## Convolutional Neural Network


### Basic CNN 
(fmap, size, kernel_size, strides, padding, activation)
- 入力 : (1, 28\*28,- , -, -)
- C1 : (32, 28\*28, 3\*3, 1, SAME, ReLU)
- C2 : (64, 14\*14, 3\*3, 2, SAME, ReLU)
- S3 : (64, 7\*7, 2\*2, 2, VALID, -)
- F4 : (-, 64\*64, -, -, -, ReLU)
- 出力 : (-, 10\*10, -, -, -, softmax)

In [0]:
import numpy as np
import os
import tensorflow as tf

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [0]:
# MNIST
height = 28
width = 28
chanels = 1
n_inputs = height * width

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, [-1, height, width, chanels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad
                        )  # (28, 28)

conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad
                        )  # (14, 14)

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")  # (7, 7)
    pool3_flat = tf.reshape(pool3, [-1, pool3_fmaps*7*7])
    
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")
    
with tf.name_scope("outputs"):
    logits = tf.layers.dense(fc1, n_outputs, name="logits")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
import time

n_epochs = 10
batch_size = 100

with tf.Session() as sess:
    init.run()
    t0 = time.time()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

        save_path = saver.save(sess, "./my_mnist_model")
        
    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

0 Train accuracy: 0.99 Test accuracy: 0.9783
1 Train accuracy: 0.97 Test accuracy: 0.9806
2 Train accuracy: 0.99 Test accuracy: 0.9849
3 Train accuracy: 1.0 Test accuracy: 0.9839
4 Train accuracy: 0.99 Test accuracy: 0.9856
5 Train accuracy: 0.99 Test accuracy: 0.9877
6 Train accuracy: 1.0 Test accuracy: 0.9856
7 Train accuracy: 1.0 Test accuracy: 0.9835
8 Train accuracy: 1.0 Test accuracy: 0.9863
9 Train accuracy: 1.0 Test accuracy: 0.9833
Total training time: 598.8s


### High Accuracy CNN
- **Dropout**
- **早期打ち切り** \\
(fmap, size, kernel_size, strides, padding, activation)
    - 入力 : (1, 28\*28,- , -, -)
    - C1 : (32, 28\*28, 3\*3, 1, SAME, ReLU)
    - C2 : (64, 28\*28, 3\*3, 1, SAME, ReLU)
    - S3 : (64, 14\*14, 2\*2, 2, VALID, -)
        - dropout C2 (dropout_rate=0.25)
    - F4 : (-, 128\*128, -, -, -, ReLU)
        - dropout F4 (dropout_rate=0.5)
    - 出力 : (-, 10\*10, -, -, -, softmax)

In [9]:
# 構築
# MNIST
height = 28
width = 28
chanels = 1
n_inputs = height * width

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 1
conv2_pad = "SAME"
conv2_dropout_rate = 0.25

pool3_fmaps = conv2_fmaps

n_fc1 = 128
fc1_dropout_rate = 0.5

n_outputs = 10

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, [-1, height, width, chanels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    training = tf.placeholder_with_default(False, shape=[], name="training")

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")  # (28, 28)
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")  # (28, 28)

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="VALID")  # (14, 14)
    pool3_flat = tf.reshape(pool3, [-1, pool3_fmaps*14*14])
    pool3_flat_drop = tf.layers.dropout(pool3_flat, conv2_dropout_rate, training=training)
    
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat_drop, n_fc1, activation=tf.nn.relu, name="fc1")  # 128
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training
    
with tf.name_scope("outputs"):
    logits = tf.layers.dense(fc1_drop, n_outputs, name="logits") # 10
    Y_proba = tf.nn.softmax(logits, name="Y_proba")
    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

SyntaxError: ignored

In [0]:
# モデルの保存と復元用の関数
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

def restore_model_params(model_params):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: tf.get_default_graph().get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    tf.get_default_session().run(assign_ops, feed_dict=feed_dict)

In [0]:
import time

# 実行
n_epochs = 1000
batch_size = 50

# 早期打ち切り
best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None

with tf.Session() as sess:
    init.run()
    t0 = time.time()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
        print("Epoch {}, train accuracy {:.4f}%, valid. accuracy {:.4f}%, valid. best loss {:.6f}%".format(
            epoch, acc_train*100, acc_val*100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            t1 = time.time()
            print("Early stopping!")
            print("Total training time: {:.1f}s".format(t1 - t0))
            break
                
    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
    print("Final accuracy on test set :", acc_test)
    save_path = saver.save(sess, "./my_mnist_model")
                    

Epoch 0, train accuracy 96.0000%, valid. accuracy 98.4400%, valid. best loss 0.058560%
Epoch 1, train accuracy 100.0000%, valid. accuracy 98.9200%, valid. best loss 0.042897%
Epoch 2, train accuracy 100.0000%, valid. accuracy 98.8800%, valid. best loss 0.042156%
Epoch 3, train accuracy 98.0000%, valid. accuracy 99.0200%, valid. best loss 0.036897%
Epoch 4, train accuracy 100.0000%, valid. accuracy 98.9800%, valid. best loss 0.035211%
Epoch 5, train accuracy 100.0000%, valid. accuracy 99.2600%, valid. best loss 0.032130%
Epoch 6, train accuracy 100.0000%, valid. accuracy 99.0400%, valid. best loss 0.029772%
Epoch 7, train accuracy 100.0000%, valid. accuracy 99.1800%, valid. best loss 0.029772%
Epoch 8, train accuracy 100.0000%, valid. accuracy 99.2000%, valid. best loss 0.029772%
Epoch 9, train accuracy 100.0000%, valid. accuracy 99.1600%, valid. best loss 0.029772%
Epoch 10, train accuracy 100.0000%, valid. accuracy 99.2800%, valid. best loss 0.029772%
Epoch 11, train accuracy 100.0000